In [1]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r requirements.txt

# Restart the runtime by ending the process.
exit()

In [2]:
from __future__ import annotations
import os

import ee
import google.auth

import folium
from serving import data
import json
from branca.element import Figure
from folium import plugins
import sys
from serving.data import get_input_image
from importlib import reload

In [3]:
current_dir = os.getcwd()
sys.path.append(current_dir+'/serving')

In [4]:
# Google cloud parameters
project = "supple-nature-370421"  # @param {type:"string"}
bucket = "vgnn"  # @param {type:"string"}
location = "us-west2-a"  # @param {type:"string"}

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"
assert bucket, "⚠️ Please provide a Cloud Storage bucket name"
assert not bucket.startswith(
    "gs://"
), f"⚠️ Please remove the gs:// prefix from the bucket name: {bucket}"
assert location, "⚠️ Please provide a Google Cloud location"

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ["GOOGLE_CLOUD_PROJECT"] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

# Initialise goofle earth engine
credentials, _ = google.auth.default()
ee.Initialize(
    credentials.with_quota_project(None),
    project=project,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

Updated property [core/project].


In [5]:
#CONSTANTS
YEAR_START=2017
YEAR_END=2023

SEASON_START=5
SEASON_END=10

CROP=1 #Corn

# True vision params
vis_params = {
    "min": 0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],}

In [6]:
# reload(data)

# Show sample filter

start_year = 2017
start_month = 4
end_month = 7
crop_type = 1 # Corn
county = "Story"

county_geom = (
    ee.FeatureCollection("TIGER/2018/Counties")
    .filter(ee.Filter.eq("NAME", county))
).geometry()

coords = county_geom.centroid().coordinates().getInfo()

s2_img_start = get_input_image(county, crop_type, start_year, start_month).clip(county_geom)
s2_img_end = get_input_image(county, crop_type, start_year, end_month).clip(county_geom)

In [7]:
# Create the map
m1 = folium.Map(coords[::-1])

# Add the CDL layer
layer_left = folium.TileLayer(
    tiles=s2_img_start.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Cropland Data Layer',
    overlay=True,
    control=True
)

layer_right = folium.TileLayer(
    tiles=s2_img_end.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Cropland Data Layer',
    overlay=True,
    control=True
)

sbs = plugins.SideBySideLayers(layer_left=layer_left, layer_right=layer_right)

layer_left.add_to(m1)
layer_right.add_to(m1)
sbs.add_to(m1)    

# Display the map
m1

In [8]:
"""
Changes in crops can be observable in the satellite images. Once can hope that the colour saturation and temporal change would hold predictive power
"""

'\nChanges in crops can be observable in the satellite images. Once can hope that the colour saturation and temporal change would hold predictive power\n'

In [26]:
img_name = f"{county}_{start_year}_{start_month}-{start_month+1}"
img_task = ee.batch.Export.image.toCloudStorage(
    image=s2_img_start,
    description=img_name,
    bucket=bucket,
    fileNamePrefix=f"images/{img_name}",
    scale=30,
    region=county_geom,
)
img_task.start()

In [8]:
img_name = f"{county}_{start_year}_{end_month}-{end_month+1}"
img_task = ee.batch.Export.image.toCloudStorage(
    image=s2_img_end,
    description=img_name,
    bucket=bucket,
    fileNamePrefix=f"images/{img_name}",
    scale=30,
    region=county_geom,
)
img_task.start()